<a href="https://colab.research.google.com/github/analyst-rhie/Kaggle/blob/main/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%A0%95%EB%8B%B5%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,\
GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold,\
learning_curve

sns.set(style='white', context='notebook', palette='deep')

1. 데이터를 로드해라

In [6]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/타이타닉/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/타이타닉/test.csv')

* 본 이상치 탐지법은 Tukey 방법에 따라 사용자가 지정한 n개 이상의 특이치를 포함하는 관측치를 골라내는 함수를 작성할 것이다.
  * detect_outliers 이름의 함수를 정의하라. 이때 들어가는 인수는 DataFrame이름, Outiler 갯수, 변수이름이다.
  * 각 변수로 부터 Q1, Q3 ,IQR = Q3 - Q1을 출력하는 변수를 위 Q1,Q3,IQR과 동일 한 이름으로 생성하라
  * outlier_step 이름에 IQR에 1.5를 곱해 outlier에 색출될 최소범위를 선택해라
  * Q1에서 outlier_step을 뺀 범위 또는 Q3에서 outlier를 더한 범위에 포함되는 index를 가져와 outlier_list_col에 저장하라
  * outlier_indices 리스트에 outlier_list_col을 extend해라
  * 모든 변수들이 위 extend 결과를 마치면, 이때 Counter 함수를 활용해 각 원소들이 몇번 반복됬는지 계산하여 outlier_indices에 덮어써라
  * outlier_indices 리스트의 값들을 outlier_indices 리스트에서 2개 이상의 이상치를 포함하고 있을때의 관측치를 선택하라. multiple_outliers에 저장하고 return해라


``` 
1. extend 함수는 리스트 끝에 반복되는 값이 리스트 일 경우 뒤에 붙여주는 역할을 함 
 -> 이떄 extend에 사용되는 리스트를 처음 지정하는 = [] 위치는 for문에 속해있으면 안된다. 조심할것!
2. 코드 예시에서는 np.percentile()을 사용했으나 Fare의 경우 결측(nan)이 존재하여 Q1, Q3결과가 nan으로 출력됨.
 -> np.nanpercentile() 으로 nan을 무시하고 출력하도록 함.
 3. Counter() 함수는 한 리스트 내에 동일한 값이 몇개 있는지 딕셔너리 형태로 출력해준다.
 4. Counter()함수로 만든 딕셔너리를 .items()로 가져오면 순서쌍으로 묶어준다. 따라서 for 문 내 두 개의 iterator를 사용하더라도 zip()을 안쓰고도 가져올 수 있게된다.
```


In [75]:
def detect_outliers(df, n ,features):
  outlier_list_sample = []
  for col in features :
    q1 = np.nanpercentile(df[col], 25)
    q3 = np.nanpercentile(df[col], 75)
    iqr = q3-q1
    outlier_step = iqr*1.5
    outlier_list = df[(df[col]< q1-outlier_step)|(df[col]>q3+outlier_step)].index
    outlier_list_sample.extend(outlier_list)
  outlier_list_sample = Counter(outlier_list_sample)
  output_list = [i for i,v in outlier_list_sample.items() if v>n]
  return output_list

example = detect_outliers(train, 1, ['Age','SibSp','Fare'])  
print(example)

[745, 27, 88, 159, 180, 201, 324, 341, 792, 846, 863]


================================================================================



In [ ]:
def detect_outliers(df, n, features):
  outlier_indices = []
  for col in features :
    print("=========================")
    print("col name : ", col)
    Q1 = np.nanpercentile(df[col], 25)
    print("Q1 : ",Q1)
    Q3 = np.nanpercentile(df[col],75)
    print("Q3 : ",Q3)
    IQR = Q3 - Q1
    print("IQR : ",IQR)
    outlier_step = 1.5*IQR
    print("outlier_step : ",outlier_step)
    outlier_list_col = df[(df[col]<Q1 - outlier_step)|(df[col]>Q3+outlier_step)].index
    print("outlier_list_col : ",outlier_list_col)
    outlier_indices.extend(outlier_list_col)
    print("outlier_indices extend 전 :",outlier_indices)
  
  outlier_indices = Counter(outlier_indices)
  print("outlier_indices extend 후 :",outlier_indices)
  print("outlier_indices.items() : ",outlier_indices.items())
  multiple_outliers = list(k for k, v in outlier_indices.items() if v>n)
  print("multiple_outliers : ",multiple_outliers)
  return multiple_outliers

outliers_to_drop = detect_outliers(train,1,['Age','SibSp','Fare'])
print("출력 된 값은 ? : ",outliers_to_drop)

In [76]:
def detect_outliers(df, n, features):
  outlier_indices = []
  for col in features :
    print("=========================")
    print("col name : ", col)
    Q1 = np.nanpercentile(df[col], 25)
    print("Q1 : ",Q1)
    Q3 = np.nanpercentile(df[col],75)
    print("Q3 : ",Q3)
    IQR = Q3 - Q1
    print("IQR : ",IQR)
    outlier_step = 1.5*IQR
    print("outlier_step : ",outlier_step)
    outlier_list_col = df[(df[col]<Q1 - outlier_step)|(df[col]>Q3+outlier_step)].index
    print("outlier_list_col : ",outlier_list_col)
    outlier_indices.extend(outlier_list_col)
    print("outlier_indices extend 전 :",outlier_indices)
  
  outlier_indices = Counter(outlier_indices)
  print("outlier_indices extend 후 :",outlier_indices)
  print("outlier_indices.items() : ",outlier_indices.items())
  multiple_outliers = list(k for k, v in outlier_indices.items() if v>n)
  print("multiple_outliers : ",multiple_outliers)
  return multiple_outliers

outliers_to_drop = detect_outliers(train,1,['Age','SibSp','Fare'])
print("출력 된 값은 ? : ",outliers_to_drop)

col name :  Age
Q1 :  20.125
Q3 :  38.0
IQR :  17.875
outlier_step :  26.8125
outlier_list_col :  Int64Index([33, 54, 96, 116, 280, 456, 493, 630, 672, 745, 851], dtype='int64')
outlier_indices extend 전 : [33, 54, 96, 116, 280, 456, 493, 630, 672, 745, 851]
col name :  SibSp
Q1 :  0.0
Q3 :  1.0
IQR :  1.0
outlier_step :  1.5
outlier_list_col :  Int64Index([  7,  16,  24,  27,  50,  59,  63,  68,  71,  85,  88, 119, 159,
            164, 171, 176, 180, 182, 201, 229, 233, 261, 266, 278, 324, 341,
            374, 386, 409, 480, 485, 541, 542, 634, 642, 683, 686, 726, 787,
            792, 813, 819, 824, 846, 850, 863],
           dtype='int64')
outlier_indices extend 전 : [33, 54, 96, 116, 280, 456, 493, 630, 672, 745, 851, 7, 16, 24, 27, 50, 59, 63, 68, 71, 85, 88, 119, 159, 164, 171, 176, 180, 182, 201, 229, 233, 261, 266, 278, 324, 341, 374, 386, 409, 480, 485, 541, 542, 634, 642, 683, 686, 726, 787, 792, 813, 819, 824, 846, 850, 863]
col name :  Fare
Q1 :  7.9104
Q3 :  31.0
IQR :  23

In [27]:
outliers_to_drop

[]